In [1]:
from transformers import pipeline

/home/wadhwa.s/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
example = "A search of the United States Food and Drug Administration's Adverse Event Reporting System identified nine cases of oxcarbazepine-associated angioedema in pediatric patients aged 16 years and younger."

In [13]:
extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(example, return_tensors=True, return_text=False)[0]["generated_token_ids"]])

In [14]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])

In [15]:
print(extracted_text[0])

<s><triplet> Food and Drug Administration <subj> Adverse Event Reporting System <obj> owner of <triplet> Adverse Event Reporting System <subj> Food and Drug Administration <obj> owned by</s>


In [16]:
print(extracted_triplets)

[{'head': 'Food and Drug Administration', 'type': 'owner of', 'tail': 'Adverse Event Reporting System'}, {'head': 'Adverse Event Reporting System', 'type': 'owned by', 'tail': 'Food and Drug Administration'}]
